In [ ]:
#@markdown GPU 종류와 사용 가능한 VRAM 확인하기
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

## 요구사항 설치

In [ ]:
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
!wget -q https://raw.githubusercontent.com/huggingface/diffusers/039958eae55ff0700cfb42a7e72739575ab341f1/scripts/convert_original_stable_diffusion_to_diffusers.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.12.0 transformers ftfy bitsandbytes gradio natsort

%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

# If precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

%pip install -q OmegaConf
# convert_original_stable_diffusion_to_diffusers.py에 OmegaConf가 필요함

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 102.9 MB 104 kB/s 


## 구글 드라이브 마운트



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## 경로 설정

In [ ]:
#@markdown 좌측 파일 탐색기에서 우클릭 - 경로 복사 할수 있음

#@markdown 구글 드라이브 폴더는 /content/drive/MyDrive에서 확인가능

#@markdown 학습 시킬 모델 파일의 경로
ORIG_MODEL_PATH = "/content/drive/MyDrive/SD/models/Stable-diffusion/animefull-final-pruned.ckpt" #@param {type:"string"}

#@markdown yaml 파일의 경로
ORIG_YAML_PATH = "/content/drive/MyDrive/SD/models/Stable-diffusion/animefull-final-pruned.yaml" #@param {type:"string"}

#@markdown 학습 완료된 모델 파일(ckpt 변환 이전)을 저장할 경로

OUTPUT_DIR = "/content/MyDrive/SD/models/Stable-diffusion/osk" #@param {type:"string"}

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

[*] Weights will be saved at /content/MyDrive/SD/models/Stable-diffusion/osk


## 모델 파일을 디퓨저 포맷으로 변환하기 (약 5분 소요)

In [ ]:
CONVERTED_MODEL_PATH = "/content/converted"

!python convert_original_stable_diffusion_to_diffusers.py \
  --checkpoint_path $ORIG_MODEL_PATH \
  --original_config_file $ORIG_YAML_PATH \
  --dump_path $CONVERTED_MODEL_PATH

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['text_projection.weight', 'vision_model.encoder.layers.7.layer_norm2.weight', 'vision_model.encoder.layers.11.self_attn.q_proj.bias', 'vision_model.encoder.layers.12.self_attn.q_proj.bias', 'vision_model.encoder.layers.21.self_attn.k_proj.weight', 'vision_model.encoder.layers.10.layer_norm1.bias', 'vision_model.encoder.layers.7.layer_norm1.weight', 'vision_model.encoder.layers.9.self_attn.v_proj.weight', 'vision_model.encoder.layers.21.layer_norm1.bias', 'vision_model.encoder.layers.0.layer_norm1.bias', 'vision_model.encoder.layers.7.layer_norm1.bias', 'vision_model.encoder.layers.6.layer_norm2.bias', 'vision_model.encoder.layers.14.layer_norm1.bias', 'vision_model.encoder.layers.20.self_attn.v_proj.bias', 'vision_model.encoder.layers.12.self_attn.k_proj.weight', 'vision_model.encoder.layers.7.mlp.fc2.weight', 'vision_model.encoder.layers.7.self_attn.q_proj.bias', 'visi

# AI Trainning

아래에 있는 표를 참조해서 메모리와 속도를 확인 가능함.
Tesla T4 GPU에서 테스트됨.


| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |


Add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.

remove `--train_text_encoder` flag to reduce memory usage further, degrades output quality.

In [ ]:
# You can also add multiple concepts here. Try tweaking `--max_train_steps` accordingly.

#@markdown 새로 만들 프롬프트
INSTANCE_PROMPT = "jakga " #@param {type:"string"}
#@markdown 새로 만들 프롬프트가 속하는 분류
CLASS_PROMPT = "girl" #@param {type:"string"}

concepts_list = [
    {
        "instance_prompt":      INSTANCE_PROMPT,
        "class_prompt":         CLASS_PROMPT,
        "instance_data_dir":    "/content/data/" + INSTANCE_PROMPT,
        "class_data_dir":       "/content/data/" + CLASS_PROMPT
    },
]

# `class_data_dir` contains regularization images
import json
import os
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)
    os.makedirs(c["class_data_dir"], exist_ok=True)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

## 이미지 파일 업로드

In [ ]:
#@markdown (이 행을 실행시 업로드 버튼이 나타남남)

import os
from google.colab import files
import shutil

for c in concepts_list:
    print(f"`{c['instance_prompt']}`에 대한 이미지 업로드 하기")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(c['instance_data_dir'], filename)
        shutil.move(filename, dst_path)

for c in concepts_list:
    print(f"`{c['class_prompt']}`에 대한 이미지 업로드 하기")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(c['class_data_dir'], filename)
        shutil.move(filename, dst_path)

## AI 훈련 시작

In [ ]:
SAMPLE_PROMPT = "\""+INSTANCE_PROMPT + " " + CLASS_PROMPT+"\""

#@markdown seed
SEED = "1337" #@param {type:"string"}

#@markdown 훈련 배치 크기
TRAIN_BATCH_SIZE = 1 #@param {type:"string"}

#@markdown 해상도
RESOLUTION = "512" #@param {type:"string"}

#@markdown 클래스 이미지 개수 (폴더 이미지가 이 숫자보다 부족하게 있으면 자동으로 추가 생성)
NUM_CLASS_IMAGES = "200" #@param {type:"string"}

#@markdown 학습률
LEARNING_RATE = "1e-6" #@param {type:"string"}

#@markdown 훈련 횟수
MAX_TRAIN_STEPS = "3000"  #@param {type:"string"}

#@markdown N회마다 훈련 결과 저장
SAVE_INTERVAL = "1000" #@param {type:"string"}

!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$CONVERTED_MODEL_PATH \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=$SEED \
  --resolution=$RESOLUTION \
  --train_batch_size=$TRAIN_BATCH_SIZE \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=$LEARNING_RATE \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=$NUM_CLASS_IMAGES \
  --sample_batch_size=1 \
  --max_train_steps=$MAX_TRAIN_STEPS \
  --save_interval=$SAVE_INTERVAL \
  --save_sample_prompt=$SAMPLE_PROMPT \
  --concepts_list="concepts_list.json"

# Reduce the `--save_interval` to lower than `--max_train_steps` to save weights from intermediate steps.
# `--save_sample_prompt` can be same as `--instance_prompt` to generate intermediate samples (saved along with weights in samples directory).

## 훈련 완료된 weights를 ckpt로 변환하기.

In [ ]:
#@markdown cpkt로 변환할 weights 폴더를 지정함

#@markdown 빈칸으로 남겨둘시 가장 마지막으로 학습된 weights가 ckpt로 변환됨됨
WEIGHTS_DIR = "" #@param {type:"string"}
if WEIGHTS_DIR == "":
    from natsort import natsorted
    from glob import glob
    import os
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

In [ ]:
#@markdown cpkt로 변환
ckpt_path = "/content/drive/MyDrive/SD/models/Stable-diffucion/osk.ckpt"

half_arg = ""
#@markdown  fp16 사용시 용량이 절반으로 줄음 (=2GB)
fp16 = True #@param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

In [ ]:
#@title (옵션) ckpt를 제외하고 전부 삭제하기

#@markdown [ ! ] Caution, Only execute if you are sure u want to delete the diffuser format weights and only use the ckpt.
import shutil
from glob import glob
import os
for f in glob(OUTPUT_DIR+os.sep+"*"):
    if f != WEIGHTS_DIR:
        shutil.rmtree(f)
        print("Deleted", f)
for f in glob(WEIGHTS_DIR+"/*"):
    if not f.endswith(".ckpt") or not f.endswith(".json"):
        try:
            shutil.rmtree(f)
        except NotADirectoryError:
            continue
        print("Deleted", f)

In [ ]:
#@title Free runtime memory
exit()